In [6]:
from collections import Counter

In [7]:
def get_slop(df2,from_date,to_date):
    datelist = pd.date_range(start=from_date, end=to_date, freq = 'M').tolist()
    dtlst = []
    df = df2.copy()
    for d_t in datelist:
        d_t = str(d_t)[0:-9]
        d = datetime.strptime(d_t, '%Y-%m-%d')
        d = d.strftime('%Y%m')
        dtlst.append(d)
    for dt in dtlst:
        df[dt] = df[dt][:-1]*10000 / df[dt][-1]
    from scipy import stats

    x = [x + 1for x in range(0,len(dtlst))]

    slopes = []
    for i in list(df.values):
        y = i[:-1]
        slope, intercept, r, p, std_err = stats.linregress(x, y)
        slopes.append(slope)
    df['slope'] = slopes
    return df

In [8]:
def data_pro(data,from_date,to_date):

    df = data[['일자','키워드']]
    df['일자'] = [str(x)[0:6] for x in df['일자']] 

    datelist = pd.date_range(start=from_date, end=to_date, freq = 'M').tolist()
    dtlst = []

    for d_t in datelist:
        d_t = str(d_t)[0:-9]
        d = datetime.strptime(d_t, '%Y-%m-%d')
        d = d.strftime('%Y%m')
        dtlst.append(d)

    # 키워드 사전 
    keyword_vocab = []
    for i in df['키워드']:
        keyword_vocab.extend(i)
    keyword_vocab = list(set(keyword_vocab))

    # 월별 카운팅 후 병합 # 병합하는 방식 수정해야함. 한달만 카운트는 현재 불가.

    tmp_df_list = []
    for dt in dtlst:
        tmp = []
        for i in df[df['일자']== dt]['키워드']:
            tmp.extend(i)
        tmp_df = pd.DataFrame(pd.Series(Counter(tmp)))
        tmp_df = tmp_df.reset_index()
        tmp_df = tmp_df.rename(columns = {0:'cnt','index':'keyword'})
        tmp_df_list.append(tmp_df)

    df = tmp_df_list[0].merge(tmp_df_list[1], on = 'keyword', how = 'outer')
    for i in range(len(tmp_df_list)):
        if i > 1:
            df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
    col = ['keyword'] + dtlst
    df.columns = col

    # 합산 값 만들기
    df.index = df['keyword']
    df = df.drop('keyword', axis = 1)
    df['tot_cnt'] = df.sum(axis = 1)
    df = df.T
    df['year_cnt'] = df.sum(axis = 1)
    df = df.T
    df

    # 결측치 처리
    df =df.fillna(0)
    return df

In [9]:
'''
하루단위로 하는 건 너무 대공사네....필요할 떄 하지
df['con_text'] = df.groupby(['일자'])['키워드'].transform(lambda x: ','.join(x))
df = df[['일자','con_text']].drop_duplicates()
'''

"\n하루단위로 하는 건 너무 대공사네....필요할 떄 하지\ndf['con_text'] = df.groupby(['일자'])['키워드'].transform(lambda x: ','.join(x))\ndf = df[['일자','con_text']].drop_duplicates()\n"

In [10]:
from datetime import datetime
import logging
import re
import numpy as np
import pandas as pd
import os

In [11]:
def DataKr(topic,from_date,to_date):
    cwd = os.getcwd()
    path = cwd +'/' + topic + '/raw'
    file_list = os.listdir(path)
    
    dfs = []
    for i in file_list:
        df = pd.read_excel(path+'/'+i)
        dfs.append(df)
        print(i)
    data = pd.concat(dfs)
    print("총 기사 건 수 : " + str(len(data)))  
    
    data = data[['뉴스 식별자','일자','제목','키워드','특성추출(가중치순 상위 50개)']]
    cluster_mode = '특성추출(가중치순 상위 50개)'
    corpus = data[cluster_mode].values.tolist()

    data_k = data[['일자']]

    for i, cp in enumerate(corpus):
        corpus[i] = cp.split(',')
    data_k['키워드'] = corpus
    
    df_k = data_pro(data_k,from_date,to_date)
    df_k.to_csv(cwd +'/' + topic + '/'+ 'Monthly_Count_Keyword {}-{}.csv'.format(from_date,to_date),encoding="utf-8-sig")

    df_ks = get_slop(df_k,from_date,to_date)
    df_ks.to_csv(cwd +'/' + topic + '/'+ 'Efficient_OLS_Keyword {}-{}.csv'.format(from_date,to_date),encoding="utf-8-sig")

In [13]:
DataKr('문제','20070101','20221231')

C:\Users\NH\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20070101-20081231.xlsx
NewsResult_20090101-20101231.xlsx
NewsResult_20110101-20121231.xlsx
NewsResult_20130101-20141231.xlsx
NewsResult_20150101-20161231.xlsx
NewsResult_20170101-20181231.xlsx
NewsResult_20190101-20191231.xlsx
NewsResult_20200101-20201231.xlsx
NewsResult_20210101-20221231.xlsx
총 기사 건 수 : 143481


C:\Users\NH\AppData\Local\Temp\ipykernel_25676\4198301622.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_k['키워드'] = corpus
C:\Users\NH\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [7]:
DataKr('식량안보','20200101','20221231')
DataKr('비용','20200101','20221231')


C:\Users\NH\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20200101-20221231 (3).xlsx
총 기사 건 수 : 1996


C:\Users\NH\AppData\Local\Temp\ipykernel_9208\4198301622.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_k['키워드'] = corpus
C:\Users\NH\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


NewsResult_20200101-20221231 (1).xlsx
총 기사 건 수 : 10386


In [ ]:

def DataEn(topic,from_date,to_date):
    

    cwd = os.getcwd()
    main_path = cwd + '/' + topic
    name = "Keyword{}-{}.pickle".format(from_date,to_date)
    name = name.replace('/','.')
    with open(main_path + '/' + 'keyword/' + name, 'rb') as f:
        data_e = pickle.load(f)

    df_k = data_pro(data_k,from_date,to_date)
    df_k.to_csv(cwd +'/' + topic + '/'+ 'Monthly_Count_Keyword {}-{}.csv'.format(from_date,to_date),encoding="utf-8-sig")

    df_ks = get_slop(df_k,from_date,to_date)
    df_ks.to_csv(cwd +'/' + topic + '/'+ 'Efficient_OLS_Keyword {}-{}.csv'.format(from_date,to_date),encoding="utf-8-sig")